In [349]:
import numpy as np
import pandas as pd
import random
import os
import copy
import gym
from gym import Env
from gym.spaces import Discrete, Box

### Defining some commonly used constants

In [350]:
SEED = 42

MAX_STEPS = 10

ACTION_SPACE = ['No anemia', 'Vitamin B12/Folate deficiency anemia', 'Unspecified anemia', 'Anemia of chronic disease', 
'Iron deficiency anemia', 'Hemolytic anemia', 'Aplastic anemia', 'Inconclusive diagnosis', 'hemoglobin', 'ferritin', 'ret_count',
'segmented_neutrophils', 'tibc', 'mcv', 'serum_iron', 'rbc', 'gender', 'creatinine', 'cholestrol', 'copper', 'ethanol', 'folate', 
'glucose', 'hematocrit', 'tsat']

CLASS_DICT = {'No anemia': 0, 'Vitamin B12/Folate deficiency anemia': 1, 'Unspecified anemia': 2, 'Anemia of chronic disease': 3, 
'Iron deficiency anemia': 4, 'Hemolytic anemia': 5, 'Aplastic anemia': 6, 'Inconclusive diagnosis': 7}

FEATURE_NUM = len(ACTION_SPACE) - len(CLASS_DICT)

#### Assigning Seed values for code reproducability

### Defining the environment

In [351]:
# NOTE: The environment code is based from the original source code of the paper with the github link below
# https://github.com/lilly-muyama/anemia_diagnosis_pathways
class AnemiaEnv(Env):
    def __init__(self, X, Y, random=True):
        super(AnemiaEnv, self).__init__()
        self.action_space = Discrete(len(ACTION_SPACE))
        self.observation_space = Box(0, 1.5, (FEATURE_NUM,))
        self.actions = ACTION_SPACE
        self.max_steps = MAX_STEPS
        self.X = X
        self.Y = Y
        self.sample_num = len(X)
        self.idx = -1
        self.x = np.zeros((FEATURE_NUM,), dtype=np.float32)
        self.y = np.nan
        self.state = np.full((FEATURE_NUM,), -1, dtype=np.float32)
        self.num_classes = len(CLASS_DICT)
        self.episode_length = 0
        self.trajectory = []
        self.total_reward = 0
        self.random = random
        self.seed()

    # def seed(self, seed=SEED):
    #     '''
    #     Defining a seed for the environment
    #     '''
    #     self.np_random, seed = gym.utils.seeding.np_random(seed)
    #     return [seed]

    
    def step(self, action):
        '''
        A step in the environment
        '''
        if isinstance(action, np.ndarray):
            action = int(action)

        self.episode_length += 1
        reward = 0
        if self.episode_length == self.max_steps: #reached maximum steos
            reward -=1
            self.total_reward -=1
            terminated = True
            done = True
            y_actual = self.y
            y_pred = CLASS_DICT['Inconclusive diagnosis']
            is_success = True if y_actual==y_pred else False
        elif action < self.num_classes: #diagnosis action
            if action == self.y:
                reward +=1
                self.total_reward += 1
                is_success = True
            else: 
                reward -= 1
                self.total_reward -= 1
                is_success = False
            terminated = False
            done = True
            y_actual = self.y
            y_pred = action
        elif self.actions[action] in self.trajectory: #terminate episode in case of repeated action
            action = CLASS_DICT['Inconclusive diagnosis']
            terminated = True
            reward -= 1
            self.total_reward -= 1
            done=True
            y_actual = self.y
            y_pred = action
            is_success = True if y_actual==y_pred else False
        else:
            terminated = False
            reward += 0.0
            self.total_reward += 0.0
            done = False
            self.state = self.get_next_state(action-self.num_classes)
            y_actual = np.nan
            y_pred = np.nan
            is_success = None
        self.trajectory.append(self.actions[action])
        info = {'index': self.idx, 'episode_length':self.episode_length, 'reward': self.total_reward, 'y_pred': y_pred, 
                'y_actual': y_actual, 'trajectory':self.trajectory, 'terminated':terminated, 'is_success': is_success}
        return self.state, reward, done, info
            
    
    def render(self):
        '''
        A representation of the current state of the environment
        '''
        print(f'STEP {self.episode_length} for index {self.idx}')
        print(f'Current state: {self.state}')
        print(f'Total reward: {self.total_reward}')
        print(f'Trajectory: {self.trajectory}')
        
    
    def reset(self, idx=None):
        '''
        Resetting the environment
        '''
        if idx is not None:
            self.idx = idx
        elif self.random:
            self.idx = random.randint(0, self.sample_num-1)
        else:
            self.idx += 1
            if self.idx == len(self.X):
                raise StopIteration()
        self.x, self.y = self.X[self.idx], self.Y[self.idx]
        self.state = np.full((FEATURE_NUM,), -1, dtype=np.float32)
        self.trajectory = []
        self.episode_length = 0
        self.total_reward = 0
        return self.state
        
    
    def get_next_state(self, feature_idx):
        '''
        Getting the next state of the environment
        '''
        self.x = self.x.reshape(-1, FEATURE_NUM)
        x_value = self.x[0, feature_idx]
        next_state = copy.deepcopy(self.state)
        next_state[feature_idx] = x_value
        return next_state

### Creating and Training the DQN Network

In [352]:
data = pd.read_csv('data/train_set_basic.csv')

In [353]:
data.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,14.728733,-1.000000,3.170892,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,44.186200,-1.0,0
1,10.405752,9.634615,5.659537,-1.000000,-1.000000,77.413788,212.671838,4.032519,0,0.887130,96.311597,-1.0,43.218595,-1.0,83.207518,31.217256,-1.0,4
2,15.132737,358.914888,1.842252,3.797487,315.102272,80.500314,-1.000000,5.639507,0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,45.398211,-1.0,0
3,11.340169,-1.000000,1.662209,2.441767,-1.000000,97.033963,102.079062,3.506041,1,1.020527,127.281715,-1.0,20.847013,-1.0,62.210273,34.020508,-1.0,6
4,6.691485,-1.000000,3.337971,-1.000000,-1.000000,99.838438,24.119564,2.010694,0,1.957666,34.633063,-1.0,34.612121,-1.0,112.411298,20.074456,-1.0,5


In [354]:
# Check value counts of the target variable
data['label'].value_counts()

0    7200
6    6501
5    6498
1    6483
2    6454
3    6378
4    6047
7    4839
Name: label, dtype: int64

In [355]:
data.fillna(-1, inplace=True)

X_train = data.iloc[:, 0:-1]
y_train = data.iloc[:, -1]

print(X_train.head())
print(y_train.head())

X_train, y_train = np.array(X_train), np.array(y_train)

   hemoglobin    ferritin  ret_count  segmented_neutrophils        tibc  \
0   14.728733   -1.000000   3.170892              -1.000000   -1.000000   
1   10.405752    9.634615   5.659537              -1.000000   -1.000000   
2   15.132737  358.914888   1.842252               3.797487  315.102272   
3   11.340169   -1.000000   1.662209               2.441767   -1.000000   
4    6.691485   -1.000000   3.337971              -1.000000   -1.000000   

         mcv  serum_iron       rbc  gender  creatinine  cholestrol  copper  \
0  -1.000000   -1.000000 -1.000000       1   -1.000000   -1.000000    -1.0   
1  77.413788  212.671838  4.032519       0    0.887130   96.311597    -1.0   
2  80.500314   -1.000000  5.639507       0   -1.000000   -1.000000    -1.0   
3  97.033963  102.079062  3.506041       1    1.020527  127.281715    -1.0   
4  99.838438   24.119564  2.010694       0    1.957666   34.633063    -1.0   

     ethanol  folate     glucose  hematocrit  tsat  
0  -1.000000    -1.0   -1.0

In [356]:
environment = AnemiaEnv(X_train, y_train)

In [357]:
from stable_baselines3 import DQN

In [358]:
model_dqn = DQN('MlpPolicy', environment, learning_rate=0.01, gamma=1, 
                buffer_size=10, learning_starts=10, 
                train_freq=4, gradient_steps=1,exploration_fraction=1, 
                exploration_initial_eps=1.0, exploration_final_eps=0.05)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/stable_baselines3/common/vec_env/patch_gym.py:50: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  "You provided an OpenAI Gym environment. "


In [359]:
model_dqn.learn(total_timesteps=10000, log_interval=4)

In [360]:
def evaluate_dqn(dqn_model, X_test, y_test):
    test_df = pd.DataFrame(columns=['index', 'episode_length', 'reward', 'y_pred', 'y_actual', 'trajectory', 'terminated', 'is_success'])
    env = AnemiaEnv(X_test, y_test, random=False)
    count=0

    try:
        while True:
            count+=1
            obs, done = env.reset(), False
            while not done:
                action, _states = dqn_model.predict(obs, deterministic=True)
                obs, reward, done, info = env.step(action)
                if done == True:
                    test_df = test_df.append(info, ignore_index=True)
    except StopIteration:
        print(f'Finished evaluating the model on {count} samples')
    return test_df

In [361]:
test_data = pd.read_csv('data/test_set_constant.csv')   
test_data.fillna(-1, inplace=True)
X_test = test_data.iloc[:, 0:-1]
y_test = test_data.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
test_df = evaluate_dqn(model_dqn, X_test, y_test)

Finished evaluating the model on 14001 samples


In [364]:
print(f"accuracy: {test_df['is_success'].mean()}")

accuracy: 0.096
